In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import traceback

In [2]:
df = pd.read_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\foreign_news_data\Forbes_9.csv')
df

,Unnamed: 0,Unnamed: 0.1,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
0,0,1418,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square Cash: A New Place To Buy And Sell Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,NaN
1,1,1419,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square's Cash App: A New Place To Buy And Sell...,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,NaN
2,2,1582,"{'id': None, 'name': 'Forbes'}","Matei Gavril, CommunityVoice, Matei Gavril, Co...",The Financial Revolution And The Many Benefits...,Cryptocurrency and the blockchain are rapidly ...,https://www.forbes.com/sites/forbescommunicati...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-15T14:00:00Z,Nowadays when people say &amp;quot;major finan...,Forbes,Share to FacebookShare to TwitterShare to Link...
3,3,1640,"{'id': None, 'name': 'Forbes'}","Omri Barzilay, Contributor, Omri Barzilay, Con...",Bitcoin2x Fork Just Stalled - What You Need to...,Bitcoin saw plenty of internal strife in the l...,https://www.forbes.com/sites/omribarzilay/2017...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-15T15:52:00Z,Bitcoin may have enjoyed highs just a couple o...,Forbes,Share to FacebookShare to TwitterShare to Link...
4,4,1694,"{'id': None, 'name': 'Forbes'}","Roger Aitken, Contributor, Roger Aitken, Contr...",Cryptocurrency CFD Trading: Consumers Warned O...,After warning about the risks of investing in ...,https://www.forbes.com/sites/rogeraitken/2017/...,https://thumbor.forbes.com/thumbor/600x315/sma...,2017-11-15T18:02:00Z,"British financial regulator, the Financial Con...",Forbes,Share to FacebookShare to TwitterShare to Link...
...,...,...,...,...,...,...,...,...,...,...,...,...
14085,14085,744053,"{'id': None, 'name': 'Forbes'}","Susie Violet Ward, Contributor, \n Susie Viole...","SEC Delays Decision On Spot Bitcoin ETFs, Incl...",The U.S. Securities and Exchange Commission ha...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T15:30:27Z,Photo by Celal Gunes/Anadolu Agency\r\nAnadolu...,Forbes,ForbesForbes Digital AssetsRegulationSEC Delay...
14086,14086,744130,"{'id': None, 'name': 'Forbes'}","Jay Paris, Contributor, \n Jay Paris, Contribu...",Los Angeles Chargers Can’t Cover Their Disappo...,The strange tale of J.C. Jackson’s tenure with...,https://www.forbes.com/sites/jayparis/2023/09/...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T17:51:20Z,Expensive Los Angeles Chargers cornerback J.C....,Forbes,ForbesBusinessSportsMoneyLos Angeles Chargers ...
14087,14087,744241,"{'id': None, 'name': 'Forbes'}","Jarrett Wolf, Contributor, \n Jarrett Wolf, Co...",The Sam Bankman-Fried Case Is Not About Crypto...,To truly understand Sam Bankman-Fried's landma...,https://www.forbes.com/sites/jarrettwolf/2023/...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T02:29:02Z,The trial of Sam Bankman-Fried (SBF) is a clas...,Forbes,ForbesForbes Digital AssetsEditors' PickThe Sa...
14088,14088,744331,"{'id': None, 'name': 'Forbes'}","Billy Bambrough, Senior Contributor, \n Billy ...",Elon Musk And X Could Be About To Decide The F...,X chief executive Linda Yaccarino is due to me...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T11:45:40Z,"BitcoinBTC\r\n, ethereum, XRPXRP\r\n and other...",Forbes,ForbesForbes Digital AssetsXrpElon Musk And X ...


In [3]:
Forbes = df[(df['content_full'] == '') | (df['content_full'].isna())]
len(Forbes)

13542

In [6]:
Forbes['content_full'] = get_contents(Forbes)
Forbes

100%|██████████| 13542/13542 [1:08:28<00:00,  3.30it/s]  
<ipython-input-6-ce8cd1902460>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Forbes['content_full'] = get_contents(Forbes)


,Unnamed: 0,Unnamed: 0.1,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
0,0,1418,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square Cash: A New Place To Buy And Sell Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
1,1,1419,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square's Cash App: A New Place To Buy And Sell...,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
15,15,2470,"{'id': None, 'name': 'Forbes'}","Quora, Contributor, Quora, Contributor https:/...",What Is Blockchain Used For Besides Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-17T17:34:00Z,"You get any opportunity, take advantage of it....",Forbes,
18,18,2715,"{'id': None, 'name': 'Forbes'}","Peter Tchir, Contributor, Peter Tchir, Contrib...",Will The Real Bitcoin Please Stand Up,The growing Google Trend of searching for ICO ...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-19T14:24:00Z,Life is short and the impact that we all can h...,Forbes,
20,20,4088,"{'id': None, 'name': 'Forbes'}","Out of Asia, Contributor, Out of Asia, Contrib...",Putting Singapore's Dollar On Blockchain May P...,Will Singapore become the first country to ful...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-23T03:53:00Z,Success is within someone’s own well-being. It...,Forbes,
...,...,...,...,...,...,...,...,...,...,...,...,...
13791,13791,743392,"{'id': None, 'name': 'Forbes'}","Lawrence Wintermeyer, Contributor, \n Lawrence...",The Top 10 Global Crypto & Digital Assets Conf...,Crypto and digital assets conferences have bec...,https://www.forbes.com/sites/lawrencewintermey...,https://imageio.forbes.com/specials-images/ima...,2023-09-28T13:59:05Z,Which crypto and digital asset conference do I...,Forbes,
13792,13792,743505,"{'id': None, 'name': 'Forbes'}","William Skipworth, Forbes Staff, \n William Sk...",Sam Bankman-Fried Could Face 'Very Long Senten...,Sam Bankman-Fried is facing a number of crimin...,https://www.forbes.com/sites/willskipworth/202...,https://imageio.forbes.com/specials-images/ima...,2023-09-28T16:35:17Z,U.S. District Judge Lewis Kaplan told indicted...,Forbes,
13793,13793,743896,"{'id': None, 'name': 'Forbes'}","Sebastien Bardoz, Forbes Councils Member, \n S...",Enhancing Your Marketing Capabilities With 3-D...,The mix of phygital retail (physical + digital...,https://www.forbes.com/sites/forbesbusinessdev...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T11:45:00Z,Sebastien is the VP Sales North America of Wed...,Forbes,
13794,13794,743925,"{'id': None, 'name': 'Forbes'}","Danielle Chemtob, Forbes Staff, \n Danielle Ch...",Forbes Daily: Apple’s Latest Legal Fight With ...,Ron DeSantis' net worth; Recession déjà vu; IR...,https://www.forbes.com/sites/daniellechemtob/2...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T12:15:45Z,This is a published version of the Forbes Dail...,Forbes,


In [18]:
empty_rows = df[(df['content_full'] == '') | (df['content_full'].isna())]
empty_rows['content_full'] = ''
empty_rows

<ipython-input-18-9a07d8cc2ba0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_rows['content_full'] = ''


,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
2,1418,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square Cash: A New Place To Buy And Sell Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
3,1419,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square's Cash App: A New Place To Buy And Sell...,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
24,2470,"{'id': None, 'name': 'Forbes'}","Quora, Contributor, Quora, Contributor https:/...",What Is Blockchain Used For Besides Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-17T17:34:00Z,"You get any opportunity, take advantage of it....",Forbes,
28,2715,"{'id': None, 'name': 'Forbes'}","Peter Tchir, Contributor, Peter Tchir, Contrib...",Will The Real Bitcoin Please Stand Up,The growing Google Trend of searching for ICO ...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-19T14:24:00Z,Life is short and the impact that we all can h...,Forbes,
32,4088,"{'id': None, 'name': 'Forbes'}","Out of Asia, Contributor, Out of Asia, Contrib...",Putting Singapore's Dollar On Blockchain May P...,Will Singapore become the first country to ful...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-23T03:53:00Z,Success is within someone’s own well-being. It...,Forbes,
...,...,...,...,...,...,...,...,...,...,...,...
14085,744053,"{'id': None, 'name': 'Forbes'}","Susie Violet Ward, Contributor, \n Susie Viole...","SEC Delays Decision On Spot Bitcoin ETFs, Incl...",The U.S. Securities and Exchange Commission ha...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T15:30:27Z,Photo by Celal Gunes/Anadolu Agency\r\nAnadolu...,Forbes,
14086,744130,"{'id': None, 'name': 'Forbes'}","Jay Paris, Contributor, \n Jay Paris, Contribu...",Los Angeles Chargers Can’t Cover Their Disappo...,The strange tale of J.C. Jackson’s tenure with...,https://www.forbes.com/sites/jayparis/2023/09/...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T17:51:20Z,Expensive Los Angeles Chargers cornerback J.C....,Forbes,
14087,744241,"{'id': None, 'name': 'Forbes'}","Jarrett Wolf, Contributor, \n Jarrett Wolf, Co...",The Sam Bankman-Fried Case Is Not About Crypto...,To truly understand Sam Bankman-Fried's landma...,https://www.forbes.com/sites/jarrettwolf/2023/...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T02:29:02Z,The trial of Sam Bankman-Fried (SBF) is a clas...,Forbes,
14088,744331,"{'id': None, 'name': 'Forbes'}","Billy Bambrough, Senior Contributor, \n Billy ...",Elon Musk And X Could Be About To Decide The F...,X chief executive Linda Yaccarino is due to me...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T11:45:40Z,"BitcoinBTC\r\n, ethereum, XRPXRP\r\n and other...",Forbes,


In [9]:
df["url"].iloc[400]

'https://www.forbes.com/sites/cbovaird/2018/07/23/bitcoin-has-risen-more-than-33-in-a-month/'

In [5]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            # Modified this part to only get the first 'div+ article' element
            article_elements = soup.select('.current-article')
            
            
            # if not article_elements:
            #     article_elements = soup.select('.CheatBody')
                       
            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [213]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            # Get all the 'div+ article' elements
            article_elements = soup.select('.js-article-content')
                       
            # Concatenate texts of all article elements
            content = ' '.join([element.get_text() for element in article_elements])

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [223]:
Bloomberg = df[df['press'] == "Bloomberg"]
Bloomberg

,source,author,title,description,url,urlToImage,publishedAt,content,press
8,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Joel Rosenblatt,Coinbase Escalates Showdown on U.S. Tax Probe ...,"The 10,000 bitcoins that seven years ago famou...",https://www.bloomberg.com/news/articles/2017-1...,https://assets.bwbx.io/images/users/iqjWHBFdfx...,2017-11-09T11:02:42Z,"The 10,000 bitcoins that seven years ago famou...",Bloomberg
55,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Andy Mukherjee,What Keynes Knew About Bitcoin,They always said the celebrated British econom...,https://www.bloomberg.com/news/articles/2017-1...,https://assets.bwbx.io/images/users/iqjWHBFdfx...,2017-11-09T23:00:12Z,While preparing antidotes for the widespread u...,Bloomberg
184,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Edward Robinson,"Bitcoin Is No Bubble, Says Investor With $213 ...",Tim Draper was walking through a pavilion at a...,https://www.bloomberg.com/news/articles/2017-1...,https://assets.bwbx.io/images/users/iqjWHBFdfx...,2017-11-10T12:33:01Z,Tim Draper was walking through a pavilion at a...,Bloomberg
203,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Tiago Ramos Alfaro,The 'Bitcoin of U.K. Stocks' Is Up More Than 300%,Semiconductor company IQE Plc continued its up...,https://www.bloomberg.com/news/articles/2017-1...,https://assets.bwbx.io/s3/javelin/public/javel...,2017-11-10T13:30:59Z,Semiconductor company IQE Plc continued its up...,Bloomberg
233,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Bloomberg,SEC is said to prepare easier path for new exc...,A new rule would likely be welcomed by much of...,https://www.bloomberg.com/professional/blog/se...,https://data.bloomberglp.com/professional/site...,2017-11-10T14:37:58Z,This article was written by Benjamin Bain and ...,Bloomberg
...,...,...,...,...,...,...,...,...,...
674220,"{'id': 'bloomberg', 'name': 'Bloomberg'}",NaN,THIEL: Moving to Florida from Silicon Valley T...,THIEL: Moving to Florida from Silicon Valley T...,https://www.bloomberg.com/news/articles/2023-0...,https://img-s-msn-com.akamaized.net/tenant/amp...,2023-05-04T19:04:36Z,(Bloomberg) -- Billionaire tech investor Peter...,Bloomberg
675444,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Suvashree Ghosh,"Binance Again Pauses Bitcoin Withdrawals, Cite...","Binance Again Pauses Bitcoin Withdrawals, Cite...",https://www.bloomberg.com/news/articles/2023-0...,NaN,2023-05-08T01:46:13Z,Warning\r\nThis website has been reported for ...,Bloomberg
675541,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Sidhartha Shukla,Binance Moved $4.4 Billion of Bitcoin (BTC) Ac...,Binance Moved $4.4 Billion of Bitcoin (BTC) Ac...,https://www.bloomberg.com/news/articles/2023-0...,NaN,2023-05-08T07:47:21Z,Warning\r\nThis website has been reported for ...,Bloomberg
675603,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Bastian Benrath,Liechtenstein’s Government Plans to Accept Bit...,Liechtenstein’s Government Plans to Accept Bit...,https://www.bloomberg.com/news/articles/2023-0...,NaN,2023-05-08T10:10:39Z,Warning\r\nThis website has been reported for ...,Bloomberg


In [73]:
# from tqdm import tqdm
# import requests
# from bs4 import BeautifulSoup
# import traceback

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 첫 번째 선택자로부터 내용 추출
#             start_content_elements = soup.select('.mainArea .FirstEle p p')
#             start_content = start_content_elements[0].get_text() if start_content_elements else ''

#             # 두 번째 선택자로부터 내용 추출
#             article_elements = soup.select('.paywall > p')
#             article_content = ' '.join([ele.get_text() for ele in article_elements])

#             두 내용을 결합
#             combined_content = start_content + ' ' + article_content

#             contents.append(combined_content)
#         except (requests.Timeout, Exception) as e:
#             if isinstance(e, requests.Timeout):
#                 print(f"Request timed out for URL: {url}")
#             else:
#                 print(f"Error occurred: {e}")
#                 traceback.print_exc()
#             contents.append('')

#     return contents


In [232]:
url = 'https://www.businesswire.com/news/home/20190408005426/en/POSaBIT-Begins-Trading-Canadian-Securities-Exchange'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
response = requests.get(url, headers=headers)
print(response.status_code)
# soup = BeautifulSoup(response.text, "html.parser")
# soup

In [25]:
# from tqdm import tqdm
# import requests
# from bs4 import BeautifulSoup

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")
            
#             selected_elements = soup.select('.ccn-content')
            
#             # soup.select('.ccn-content')가 빈 리스트를 반환할 경우 에러 메시지 출력
#             if not selected_elements:
#                 print(f"No elements found for URL {url}.")
#                 contents.append('')
#                 continue

#             content = []
#             for element in selected_elements:
#                 content.append(element.get_text())
#                 print(element)

#             cleaned_content = ' '.join(content)
#             contents.append(cleaned_content)
#         except Exception as e:
#             print(f"Error occurred while processing URL {url}: {e}")
#             contents.append('')

#     return contents



In [88]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents

In [230]:
head = Bloomberg.head()
head['content_full'] = get_contents(head)
head

100%|██████████| 5/5 [00:02<00:00,  2.19it/s]
<ipython-input-230-67f0557365b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
8,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Joel Rosenblatt,Coinbase Escalates Showdown on U.S. Tax Probe ...,"The 10,000 bitcoins that seven years ago famou...",https://www.bloomberg.com/news/articles/2017-1...,https://assets.bwbx.io/images/users/iqjWHBFdfx...,2017-11-09T11:02:42Z,"The 10,000 bitcoins that seven years ago famou...",Bloomberg,
55,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Andy Mukherjee,What Keynes Knew About Bitcoin,They always said the celebrated British econom...,https://www.bloomberg.com/news/articles/2017-1...,https://assets.bwbx.io/images/users/iqjWHBFdfx...,2017-11-09T23:00:12Z,While preparing antidotes for the widespread u...,Bloomberg,
184,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Edward Robinson,"Bitcoin Is No Bubble, Says Investor With $213 ...",Tim Draper was walking through a pavilion at a...,https://www.bloomberg.com/news/articles/2017-1...,https://assets.bwbx.io/images/users/iqjWHBFdfx...,2017-11-10T12:33:01Z,Tim Draper was walking through a pavilion at a...,Bloomberg,
203,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Tiago Ramos Alfaro,The 'Bitcoin of U.K. Stocks' Is Up More Than 300%,Semiconductor company IQE Plc continued its up...,https://www.bloomberg.com/news/articles/2017-1...,https://assets.bwbx.io/s3/javelin/public/javel...,2017-11-10T13:30:59Z,Semiconductor company IQE Plc continued its up...,Bloomberg,
233,"{'id': 'bloomberg', 'name': 'Bloomberg'}",Bloomberg,SEC is said to prepare easier path for new exc...,A new rule would likely be welcomed by much of...,https://www.bloomberg.com/professional/blog/se...,https://data.bloomberglp.com/professional/site...,2017-11-10T14:37:58Z,This article was written by Benjamin Bain and ...,Bloomberg,


In [20]:
# Adding the content to the DataFrame
empty_rows['content_full'] = get_contents(empty_rows)
empty_rows

100%|██████████| 13998/13998 [1:11:12<00:00,  3.28it/s]
<ipython-input-20-2f7c53221b6c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_rows['content_full'] = get_contents(empty_rows)


,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
2,1418,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square Cash: A New Place To Buy And Sell Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
3,1419,"{'id': None, 'name': 'Forbes'}","Laura Shin, Forbes Staff, Laura Shin, Forbes S...",Square's Cash App: A New Place To Buy And Sell...,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-15T02:11:00Z,My definition of success is making an impact. ...,Forbes,
24,2470,"{'id': None, 'name': 'Forbes'}","Quora, Contributor, Quora, Contributor https:/...",What Is Blockchain Used For Besides Bitcoin?,Forbes Welcome page -- Forbes is a global medi...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-17T17:34:00Z,"You get any opportunity, take advantage of it....",Forbes,
28,2715,"{'id': None, 'name': 'Forbes'}","Peter Tchir, Contributor, Peter Tchir, Contrib...",Will The Real Bitcoin Please Stand Up,The growing Google Trend of searching for ICO ...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-19T14:24:00Z,Life is short and the impact that we all can h...,Forbes,
32,4088,"{'id': None, 'name': 'Forbes'}","Out of Asia, Contributor, Out of Asia, Contrib...",Putting Singapore's Dollar On Blockchain May P...,Will Singapore become the first country to ful...,https://www.forbes.com/forbes/welcome/?toURL=h...,NaN,2017-11-23T03:53:00Z,Success is within someone’s own well-being. It...,Forbes,
...,...,...,...,...,...,...,...,...,...,...,...
14085,744053,"{'id': None, 'name': 'Forbes'}","Susie Violet Ward, Contributor, \n Susie Viole...","SEC Delays Decision On Spot Bitcoin ETFs, Incl...",The U.S. Securities and Exchange Commission ha...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T15:30:27Z,Photo by Celal Gunes/Anadolu Agency\r\nAnadolu...,Forbes,
14086,744130,"{'id': None, 'name': 'Forbes'}","Jay Paris, Contributor, \n Jay Paris, Contribu...",Los Angeles Chargers Can’t Cover Their Disappo...,The strange tale of J.C. Jackson’s tenure with...,https://www.forbes.com/sites/jayparis/2023/09/...,https://imageio.forbes.com/specials-images/ima...,2023-09-29T17:51:20Z,Expensive Los Angeles Chargers cornerback J.C....,Forbes,
14087,744241,"{'id': None, 'name': 'Forbes'}","Jarrett Wolf, Contributor, \n Jarrett Wolf, Co...",The Sam Bankman-Fried Case Is Not About Crypto...,To truly understand Sam Bankman-Fried's landma...,https://www.forbes.com/sites/jarrettwolf/2023/...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T02:29:02Z,The trial of Sam Bankman-Fried (SBF) is a clas...,Forbes,
14088,744331,"{'id': None, 'name': 'Forbes'}","Billy Bambrough, Senior Contributor, \n Billy ...",Elon Musk And X Could Be About To Decide The F...,X chief executive Linda Yaccarino is due to me...,https://www.forbes.com/sites/digital-assets/20...,https://imageio.forbes.com/specials-images/ima...,2023-09-30T11:45:40Z,"BitcoinBTC\r\n, ethereum, XRPXRP\r\n and other...",Forbes,


In [210]:
head['content_full'].iloc[0]

"\n\n\n\n\n\n\n\r\n\r\n                        An obscure cloud service company has been providing state-sponsored hackers with internet services to spy on and extort their victims, cyber security researchers at Halcyon said.\n\nThe company, Cloudzy, had been leasing server space and reselling it to no fewer than 17 different state-sponsored ...\r\n\r\n                        \r\n                            \nHi! You've reached one of our premium articles. This is available exclusively to subscribers.\nIt's free to register, and only takes a few minutes.\nOnce you sign up you'll have unlimited access to the full catalogue of Australia's best business IT content, as well as a daily news bulletin delivered straight to your inbox.\nRegister now\nAlready have an account? Log in to read this article.\n\n"

In [200]:
head['url'].iloc[0]

'https://blogs.sap.com/2023/09/15/preparing-for-pcoe-sap-commerce-support-consultant/comment-page-1/#comments'

In [152]:
with3

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
382417,"{'id': None, 'name': 'Theregister.com'}",Robin Birtstone,Make the move to managed databases,And free your DBAs to focus ‘on the valuable s...,https://www.theregister.com/2021/08/02/move_to...,https://regmedia.co.uk/2020/04/24/shutterstock...,2021-08-02T07:30:10Z,Sponsored Have you hugged a database administr...,Theregister.com,\nSponsored Have you hugged a database adminis...
383189,"{'id': None, 'name': 'Theregister.com'}",Laura Dobberstein,US SEC chair calls for crypto regulation,"Says digi-bucks are 'rife with fraud, scams, a...",https://www.theregister.com/2021/08/04/sec_cha...,https://regmedia.co.uk/2017/11/10/currency_shu...,2021-08-04T06:01:03Z,US Securities and Exchange Commission (SEC) ch...,Theregister.com,\nUS Securities and Exchange Commission (SEC) ...
383373,"{'id': None, 'name': 'Theregister.com'}",Tim Anderson,Microsoft suspends free trials for Windows 365...,Potential customers or freeloaders to blame? A...,https://www.theregister.com/2021/08/04/windows...,https://regmedia.co.uk/2021/08/04/win365.jpg,2021-08-04T14:32:12Z,"A free two-month trial for Windows 365, a virt...",Theregister.com,"\nA free two-month trial for Windows 365, a vi..."
383648,"{'id': None, 'name': 'Theregister.com'}",Paul Kunert,Ch-ch-ch-Chia! HDD sales soar to record levels...,Knock-on effect for NAS and SSD devices too Th...,https://www.theregister.com/2021/08/05/chia_hd...,https://regmedia.co.uk/2021/07/01/shutterstock...,2021-08-05T09:32:12Z,The Chia cryptocurrency craze is fuelling reco...,Theregister.com,\nThe Chia cryptocurrency craze is fuelling re...
383969,"{'id': None, 'name': 'Theregister.com'}",Katyanna Quach,US proposes tracking digital cash and taxing i...,Uncle Sam wants digital asset brokers to log c...,https://www.theregister.com/2021/08/06/bitcoin...,https://regmedia.co.uk/2021/06/23/shutterstock...,2021-08-06T00:50:45Z,US senators revised a section of the bipartisa...,Theregister.com,\nUS senators revised a section of the biparti...
...,...,...,...,...,...,...,...,...,...,...
439158,"{'id': None, 'name': 'Theregister.com'}",Robin Birtstone,How ransomware gangs went pro,They're developing new techniques 'in every ar...,https://www.theregister.com/2022/01/05/how_ran...,https://regmedia.co.uk/2021/04/08/shutterstock...,2022-01-05T08:30:08Z,Paid Feature Ransomware has come a long way si...,Theregister.com,\nPaid Feature Ransomware has come a long way ...
439327,"{'id': None, 'name': 'Theregister.com'}",Gareth Corfield,Remember Norton 360's bundled cryptominer? Irr...,Disable anti-tamper features first and you'll ...,https://www.theregister.com/2022/01/05/norton_...,https://regmedia.co.uk/2021/07/15/shutterstock...,2022-01-05T15:56:10Z,Norton antivirus's inbuilt cryptominer has re-...,Theregister.com,\nUpdated Norton antivirus's inbuilt cryptomin...
439471,"{'id': None, 'name': 'Theregister.com'}",Liam Proven,Mozilla founder blasts browser maker for accep...,"You're spitting fire, fox\nA few days ago, Moz...",https://www.theregister.com/2022/01/05/mozilla...,https://regmedia.co.uk/2021/01/26/shutterstock...,2022-01-05T21:20:43Z,"A few days ago, Mozilla Foundation invited net...",Theregister.com,"\nUpdated A few days ago, Mozilla Foundation i..."
439851,"{'id': None, 'name': 'Theregister.com'}",Gareth Corfield,Bitcoin 'inventor' will face forgery claims ov...,COPA load of that A man who claims to be the s...,https://www.theregister.com/2022/01/06/craig_w...,https://regmedia.co.uk/2016/06/06/bitcoinsstac...,2022-01-06T15:57:05Z,A man who claims to be the secret inventor of ...,Theregister.com,\nA man who claims to be the secret inventor o...


In [28]:
with1 = TheGlobeAndMail[TheGlobeAndMail['content_full'] != '']
with2 = without[without['content_full'] != '']
len(with1)

436

In [21]:
with1 = empty_rows[empty_rows['content_full'] != '']
len(with1)

65

In [7]:
empty_rows = Forbes[Forbes['content_full'] == '']
len(empty_rows)

# print(len(CryptoSlate[CryptoSlate['content_full'] == '']))

13539

In [15]:
without = empty_rows

In [14]:
empty_rows['url'].iloc[230]

'https://www.forbes.com/sites/andrewrossow/2018/07/02/hollywood-film-producer-andrea-iervolino-encourages-hollywood-to-go-blockchain-with-tatatu-platform/'

In [24]:
empty_rows = without[without['content_full'] == '']
len(empty_rows)

167

In [27]:
without['url'].iloc[100]

'https://www.theglobeandmail.com/globe-investor/inside-the-market/before-the-bell-what-every-canadian-investor-knows-today/article37381229/'

In [101]:
empty_rows['url'].iloc[0]

'http://www.moneycontrol.com/news/business/news-live-psbs-want-large-banks-to-lend-to-corporates-retail-only-smaller-banks-says-report-2436469.html'

In [161]:
with1

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
253348,"{'id': None, 'name': 'Theregister.com'}",Robbie Harb,Galaxy S20 security is already old hat as Sams...,Passport-grade chippery to help mobile devices...,https://www.theregister.com/2020/05/26/samsung...,https://regmedia.co.uk/2020/05/26/samsung_mobi...,2020-05-26T10:32:11Z,Samsung will launch a new standalone turnkey s...,Theregister.com,\nSamsung will launch a new standalone turnkey...
253665,"{'id': None, 'name': 'Theregister.com'}",Matthew Hughes,5G mast set aflame in leafy Liverpool district...,You know sick people call ambulances using mob...,https://www.theregister.com/2020/05/27/5g_mast...,https://regmedia.co.uk/2017/10/23/shutterstock...,2020-05-27T16:41:17Z,Another 5G mast has gone up in flames in Liver...,Theregister.com,\nAnother 5G mast has gone up in flames in Liv...
253784,"{'id': None, 'name': 'Theregister.com'}",Dan Robinson,Why zero trust security needs strong hardware ...,Intel vPro platform provides hardware level of...,https://www.theregister.com/2020/05/28/strong_...,https://regmedia.co.uk/2017/10/20/security_shu...,2020-05-28T06:00:11Z,Sponsored Sometimes it feels like the hackers ...,Theregister.com,\nSponsored Sometimes it feels like the hacker...
254254,"{'id': None, 'name': 'Theregister.com'}",Shaun Nichols,Cisco hacked: Six backend servers used by cust...,Plus other news from infosec land this week\nR...,https://www.theregister.com/2020/05/31/cisco_s...,https://regmedia.co.uk/2019/08/21/cisco_shutte...,2020-05-31T09:32:11Z,Roundup Six Cisco-operated servers were hacked...,Theregister.com,\nRoundup Six Cisco-operated servers were hack...
254320,"{'id': None, 'name': 'Theregister.com'}",David Gordon,The inevitable coronavirus-inspired cyber-atta...,"They’re not very sophisticated, but they’re wo...",https://www.theregister.com/2020/06/01/coronav...,https://regmedia.co.uk/2017/03/07/shutterstock...,2020-06-01T06:00:05Z,Webcast Youve probably had the COVID-19 corona...,Theregister.com,\nWebcast You’ve probably had the COVID-19 cor...
...,...,...,...,...,...,...,...,...,...,...
572144,"{'id': None, 'name': 'Theregister.com'}",Laura Dobberstein,Microsoft revising licenses and prices in Kore...,PLUS: Tencent's edgy JV; US/China space spat; ...,https://www.theregister.com/2022/11/06/microso...,https://regmedia.co.uk/2021/08/19/shutterstock...,2022-11-06T22:59:05Z,Asia In Brief Microsoft has informed its chann...,Theregister.com,\nAsia In Brief Microsoft has informed its cha...
572211,"{'id': None, 'name': 'Theregister.com'}",Laura Dobberstein,China is likely stockpiling and deploying vuln...,Increase in espionage and cyberattacks since l...,https://www.theregister.com/2022/11/07/china_s...,https://regmedia.co.uk/2021/10/29/shutterstock...,2022-11-07T07:56:05Z,Microsoft has asserted that China's offensive ...,Theregister.com,\nMicrosoft has asserted that China's offensiv...
572326,"{'id': None, 'name': 'Theregister.com'}",Jessica Lyons Hardcastle,Can confidential computing stop the next crypt...,Tech giants and startups rush into the next bi...,https://www.theregister.com/2022/11/07/confide...,https://regmedia.co.uk/2015/07/13/crypto_32534...,2022-11-07T13:30:09Z,The theft of billions of dollars in cryptocurr...,Theregister.com,\nAmid the theft of billions of dollars in cry...
572568,"{'id': None, 'name': 'Theregister.com'}",Brandon Vigliarolo,Feds find Silk Road thief's $1b+ Bitcoin stash...,Uncle Sam follows the money ... all the way to...,https://www.theregister.com/2022/11/07/us_bitc...,https://regmedia.co.uk/2021/06/23/shutterstock...,2022-11-07T22:28:44Z,"A crook who stole more than 50,000 Bitcoins fr...",Theregister.com,"\nA crook who stole more than 50,000 Bitcoins ..."


In [30]:
TheGlobeAndMail = pd.concat([with1, with2, empty_rows], ignore_index=True)

TheGlobeAndMail

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
0,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",NaN,Streetwise newsletter: Manulife eyeing options...,Plus:AT&T ready to fight U.S. in court over Ti...,https://www.theglobeandmail.com/report-on-busi...,https://beta.theglobeandmail.com/resizer/2zZlm...,2017-11-09T23:29:18Z,"Canadian insurer Manulife Financial Corp is ""l...",The Globe And Mail,Manulife’s Bloor Street office in Toronto is s...
1,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",IAN McGUGAN,Rise of cryptocurrencies proves that nothing g...,Since gold and bitcoin appeal to similar audie...,https://www.theglobeandmail.com/globe-investor...,https://beta.theglobeandmail.com/resizer/kJ-Tb...,2017-11-09T23:31:18Z,There's an old rule in investing that insists ...,The Globe And Mail,ShareThere's an old rule in investing that ins...
2,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",NaN,"Why starting from scratch can be a benefit, a ...",A roundup of investment ideas for active inves...,https://www.theglobeandmail.com/globe-investor...,https://beta.theglobeandmail.com/resizer/VDDmH...,2017-11-10T20:36:22Z,It's a bit disquieting that he used the rise o...,The Globe And Mail,Sometimes starting from scratch can be the bes...
3,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",NIALL McGEE,"No earnings, no problem: Investors buy Giustra...",A gold penny stock was turned into a cryptocur...,https://www.theglobeandmail.com/report-on-busi...,https://beta.theglobeandmail.com/resizer/hn3fz...,2017-11-10T22:52:44Z,A shift from gold to the blockchain has paid o...,The Globe And Mail,"Billionaire Frank Giustra, chairman and CEO r ..."
4,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",NaN,Streetwise newsletter: Last week’s best reads:...,PLUS: Hot oil markets ignite speculation aroun...,https://www.theglobeandmail.com/report-on-busi...,https://beta.theglobeandmail.com/resizer/T_Hc-...,2017-11-10T23:06:40Z,A bitcoin craze sweeps Bay Street – and looks ...,The Globe And Mail,Canadian bank headquarters stand on Bay Street...
...,...,...,...,...,...,...,...,...,...,...
669,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",Trevor Hunnicutt and Jennifer Ablan,DoubleLine’s Gundlach sees U.S. 10-year Treasu...,‘ Bond King’ says conviction is high that the ...,https://www.theglobeandmail.com/globe-investor...,https://www.theglobeandmail.com/resizer/Bw5EXJ...,2018-03-14T13:18:55Z,"Jeffrey Gundlach, the chief executive of Doubl...",The Globe And Mail,
670,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",Anthony Boadle and David Lawder,"G20 sees need for ‘dialogue,’ fails to defuse ...",Finance ministers and central bankers of the w...,https://www.theglobeandmail.com/report-on-busi...,https://www.theglobeandmail.com/resizer/Cr8KYQ...,2018-03-21T09:50:24Z,The world's financial leaders rejected protect...,The Globe And Mail,
671,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",Anna Irrera And Michelle Price,Cryptocurrency issuers clean up as SEC vows to...,The issuers are increasingly filing their ‘ini...,https://www.theglobeandmail.com/report-on-busi...,https://www.theglobeandmail.com/resizer/WlxfL0...,2018-03-22T11:45:52Z,(Repeats March 21 item for wider distribution)...,The Globe And Mail,
672,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",Alexandra Jaffe,Joe Biden nears final decision on running mate...,<ol><li>Joe Biden nears final decision on runn...,https://www.theglobeandmail.com/world/article-...,https://www.theglobeandmail.com/resizer/HQftNt...,2020-08-01T17:14:32Z,Democratic presidential candidate former Vice ...,The Globe And Mail,


In [219]:
Euronews.to_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\foreign_news_data\Euronews_196.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content